In [41]:
# imports
import torch
from torch import nn 
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor

In [42]:
# Download and state traning sets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [43]:
batch_size = 64

#create DataLoaders
train_DT = DataLoader(training_data, batch_size=batch_size)
test_DT = DataLoader(test_data, batch_size=batch_size)

for X, y in test_DT:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y : {y.shape} {y.dtype}")
    break 


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64


In [44]:
# Short Neural Network

device=("cpu")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [45]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [46]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [47]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [49]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------")
    train(train_DT, model, loss_fn, optimizer)
    test(test_DT, model, loss_fn)
print("Done")

Epoch 1
------------------------
loss: 1.187942 [   64/60000]
loss: 1.182702 [ 6464/60000]
loss: 1.006158 [12864/60000]
loss: 1.134453 [19264/60000]
loss: 1.007271 [25664/60000]
loss: 1.036074 [32064/60000]
loss: 1.070305 [38464/60000]
loss: 1.007572 [44864/60000]
loss: 1.050606 [51264/60000]
loss: 0.983729 [57664/60000]
Test error: 
 Accuracy: 65.3%, Avg loss: 0.998821 

Epoch 2
------------------------
loss: 1.064424 [   64/60000]
loss: 1.079859 [ 6464/60000]
loss: 0.888793 [12864/60000]
loss: 1.040619 [19264/60000]
loss: 0.918229 [25664/60000]
loss: 0.938319 [32064/60000]
loss: 0.990264 [38464/60000]
loss: 0.930696 [44864/60000]
loss: 0.969996 [51264/60000]
loss: 0.915720 [57664/60000]
Test error: 
 Accuracy: 66.4%, Avg loss: 0.924272 

Epoch 3
------------------------
loss: 0.974130 [   64/60000]
loss: 1.009172 [ 6464/60000]
loss: 0.805976 [12864/60000]
loss: 0.973928 [19264/60000]
loss: 0.859556 [25664/60000]
loss: 0.867196 [32064/60000]
loss: 0.933634 [38464/60000]
loss: 0.879634